## Задание


Нужно решить задачу классификации наличия болезни сердца у пациентов наиболее эффективно. Целевая переменная – наличие болезни сердца (HeartDisease). Она принимает значения 0 или 1 в зависимости от отсутствия или наличия болезни соответственно. Подробное описание признаков можно прочесть в описании датасета на сайте.

In [ ]:
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

In [80]:
import pandas as pd

In [82]:
# подгружаем данные
df=pd.read_csv('heart.csv')
df

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,M,TA,110,264,0,Normal,132,N,1.2,Flat,1
914,68,M,ASY,144,193,1,Normal,141,N,3.4,Flat,1
915,57,M,ASY,130,131,0,Normal,115,Y,1.2,Flat,1
916,57,F,ATA,130,236,0,LVH,174,N,0.0,Flat,1


In [84]:
X = df.drop(columns = ['HeartDisease'])
y=df['HeartDisease']

In [94]:
# переведем категориальные переменные к значениям 1 и 0
X = pd.get_dummies(X)

In [ ]:
# разделим датасет на тренировочную итестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [108]:
# подберем лучшие гиперпараметры моделей 
models=[
    {'name':'Lr',"model": LogisticRegression()  , 'params':{'C':[0.1,0.2,0.3,0.5,0.7,1], 'penalty':['l1', 'l2']}},
      {'name':'RF',"model": RandomForestClassifier(), 'params':{'n_estimators':[10,25,50,100,150,200], 'criterion':['gini', 'entropy'], 'max_depth':[3,5,7,9,11]}},
    {'name':'SVC',"model": SVC(), 'params':{'kernel':['linear', 'poly', 'rbf', 'sigmoid'], 'gamma':['scale', 'auto']}}, 
    {'name':'DT',"model": DecisionTreeClassifier(), 'params':{'criterion':['gini', 'entropy'], 'max_depth':[3,5,7,9,11]}},
    {'name':'KN',"model": KNeighborsClassifier(), 'params':{'n_neighbors':list(range(1,30)),'weights': ['uniform', 'distance'], 'p':[1,2,3]}}
]

res=[]
for v in  models:
    res.append((v['name'], GridSearchCV(v['model'], v['params'], cv=10).fit(X_train, y_train)))

In [109]:
for r in res:
    print(r[0], r[1].best_score_, r[1].best_params_)

Lr 0.8679192891521657 {'C': 0.2, 'penalty': 'l2'}
RF 0.8869677897075157 {'criterion': 'gini', 'max_depth': 9, 'n_estimators': 150}
SVC 0.8720103665309145 {'gamma': 'scale', 'kernel': 'linear'}
DT 0.8555349870418365 {'criterion': 'entropy', 'max_depth': 7}
KN 0.8843576453165495 {'n_neighbors': 15, 'p': 1, 'weights': 'distance'}


In [110]:
#скорректируем сетку параметров по некоторым моделям на более близкие к лучшим гиперпараметрам,
#полученным на предыдущем шаге
models=[
 {'name':'Lr',"model": LogisticRegression()  , 'params':{'C':[0.05,0.15,0.25,0.35,0.55,0.75,1], 'penalty':['l1', 'l2']}},
 {'name':'RF',"model": RandomForestClassifier(), 'params':{'n_estimators':[150,200,300], 'criterion':[ 'entropy'], 'max_depth':[8,9,10,11,12]}},
 {'name':'DT',"model": DecisionTreeClassifier(), 'params':{'criterion':[ 'entropy'], 'max_depth':[6,7,8]}}
]

res=[]
for v in  models:
    res.append((v['name'], GridSearchCV(v['model'], v['params'], cv=10).fit(X_train, y_train)))

In [111]:
for r in res:
    print(r[0], r[1].best_score_, r[1].best_params_)

Lr 0.8692706405035171 {'C': 0.15, 'penalty': 'l2'}
RF 0.8870048130322102 {'criterion': 'entropy', 'max_depth': 10, 'n_estimators': 200}
DT 0.8473528322843391 {'criterion': 'entropy', 'max_depth': 7}


Удалось добиться лучших результатов на модели Случайного леса - score 0.887